#### 번역 및 정리 http://www.erogol.com/duplicate-question-detection-deep-learning/

Duplicate Question Detection with Deep Learning on Quora Dataset

Quora는 최근 그들이 공개한 첫번째 공개 데이터 세트를 발표했다. 여기에는 중복 여부를 나타내는 레이블 열이 있는 404351 질문 쌍이 포함되어 있다.  이 포스트에서는 이 데이터 집합을 조사하여 적어도 심층 학습이 있는 기준 방법을 제안하고 싶습니다.

제안된 방법 외에도, 팬더, 겐심, 스페이스, 케라스를 이용하는 방법을 보여 주는 몇가지 예가 있다. 일반 코드는 깃허브(https://github.com/erogol/QuoraDQBaseline) 을 확인해 보세요.

##### 데이터 쿼리
여기에는 255045-(비-중복)및 149306-양(중복)의 인스턴스가 있습니다. 이것은 문제의 본질을 고려할 때 클래스 불균형을 유발하지만, 실제 시나리오에서 부정적인 인스턴스를 더 많이 예상할 수 있기 때문에 당신의 맥스 모델에 동일한 데이터 바이어스를 유지하는 것이 합리적이다.

자료를 분석할 때, 가장 짧은 질문은 1자 길이이며 가장 긴 질문은 1169자이다. 만약 쌍들이 10문자보다 짧으면, 그들은 의미가 없어지고, 그래서 나는 그런 쌍들을 제거한다.  평균 길이는 59이고, std는 32이다.

다른 두개의 컬럼"q1id"와 ".id"가 있지만 다른 행에 사용된 동일한 질문에 다른 ID가 있기 때문에 그것들이 어떻게 유용한지 정말로 모르겠다.

일부 레이블, 특히 복제 레이블에 대해서는 해당되지 않습니다. 어쨌든, 나는 그 라벨에 의지하고 힘든 수동 작업 때문에 가지 치기를 미루기로 결정했다

##### 제안 방법
###### 질문을 벡터로 변환
여기서, 저는 Word2vec을 사용해서 각 질문을 의미 벡터로 변환한 다음, Siamese네트워크를 쌓아서 쌍이 중복되는지 탐지할 계획입니다.
Word2vec은 일반적으로 300개의 차원으로 이루어진 벡터 공간에 단어를 심어 넣는 유사한 알고리즘에 사용되는 일반적인 용어이다.  이 벡터들은 의미론과 심지어 다른 단어들 사이의 유사점을 캡처한다. 그 유명한 예는 다음과 같다.

왕-남자+여자=여왕

word2vec 벡터는 유용한 용도에 사용될 수 있다. 더욱 사전에 적용할 수 있도록 의미론적 단어 유사성을 계산하거나 문서를 분류하거나 이러한 벡터를 현재의 신경 네트워크에 입력할 수 있다.

이 영역에는 잘 알려 진 두개의 알고리즘이 있다. 하나는 구글의 네트워크 아키텍처이며, 이것은 주어진 윈도우 크기에 대한 목표 단어의 주변 단어들을 예측하려고 노력함으로써 표현을 배운다. GLOVE 는 공동 비밀 매트릭스에 의존하는 또 하나의 메토스이다. 글로브는 훈련시키기 쉽고 여러분의 어휘의 밖에 새로운 단어를 추가하는 것은 유연하다. 본 자습서를 참조하여 보다 자세한 내용을 확인하고 이 뛰어난 사용 사례 Sense2Vec를 확인할 수 있습니다.

우리는 여전히 싱글 턴 질문 표현을 위해 단어 벡터를 결합시키는 방법이 필요하다. 한가지 간단한 대안은 각각의 질문에 대한 모든 단어의 벡터의 평균을 취하는 것이다. 이것은 단순하지만 문서 분류에 정말 효과적인 방법이며 이 문제에도 효과가 있을 것으로 기대한다.   또한, 각 단어에 대해 정의된 TF-IDF점수를 사용하여 평균 벡터 표현을 강화할 수 있습니다. 우리는 이러한 점수를 사용해서 단어 벡터의 가중 평균을 적용한다. 그것은 식별력 있는 단어의 중요성을 강조하며, 많은 질문들에 의해 공유되는 불필요하고 빈번한 단어들을 피하게 한다.

##### Siamese 네트워크
나는 이전 포스트에서 Siamese네트워크를 묘사했다. 요컨대, 그것은 양쪽에서 두개의 입력을 취하는 양방향 네트워크 아키텍처이다. 유사한 항목이 계약되고 다른 항목이 학습된 공간에 분산되는 공간에 데이터를 투영합니다. 이것은 네트워크가 매개 변수를 공유하기 때문에 컴퓨터 작업에 효과적이다.

#### 데이터 불러오기

In [1]:
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
train_df['question1'] = train_df['question1'].apply(str)
train_df['question2'] = train_df['question2'].apply(str)

In [4]:
len(train_df)

404290

#### 토큰화 하기!! ( 한글은 Konlpy로 대체 하자 )

In [5]:
import gensim

questions = list(train_df['question1']) + list(train_df['question2'])

# tokenize
c = 0
for question in tqdm(questions):
    ## 추후에는 Konlpy로 대체 가능할것으로 보임
    questions[c] = list(gensim.utils.tokenize(question, deacc=True, lower=True))
    c += 1
questions

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
100%|███████████████████████████████████████████████████████████████████████| 808580/808580 [00:36<00:00, 22064.56it/s]


[['what',
  'is',
  'the',
  'step',
  'by',
  'step',
  'guide',
  'to',
  'invest',
  'in',
  'share',
  'market',
  'in',
  'india'],
 ['what',
  'is',
  'the',
  'story',
  'of',
  'kohinoor',
  'koh',
  'i',
  'noor',
  'diamond'],
 ['how',
  'can',
  'i',
  'increase',
  'the',
  'speed',
  'of',
  'my',
  'internet',
  'connection',
  'while',
  'using',
  'a',
  'vpn'],
 ['why',
  'am',
  'i',
  'mentally',
  'very',
  'lonely',
  'how',
  'can',
  'i',
  'solve',
  'it'],
 ['which',
  'one',
  'dissolve',
  'in',
  'water',
  'quikly',
  'sugar',
  'salt',
  'methane',
  'and',
  'carbon',
  'di',
  'oxide'],
 ['astrology',
  'i',
  'am',
  'a',
  'capricorn',
  'sun',
  'cap',
  'moon',
  'and',
  'cap',
  'rising',
  'what',
  'does',
  'that',
  'say',
  'about',
  'me'],
 ['should', 'i', 'buy', 'tiago'],
 ['how', 'can', 'i', 'be', 'a', 'good', 'geologist'],
 ['when', 'do', 'you', 'use', 'シ', 'instead', 'of', 'し'],
 ['motorola',
  'company',
  'can',
  'i',
  'hack',
  'my'

In [6]:
model = gensim.models.Word2Vec(questions, size=300, workers=-1, iter=10, negative=20)

In [7]:
model.init_sims(replace=True)

In [8]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
print("Number of tokens in Word2Vec:", len(w2v.keys()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


Number of tokens in Word2Vec: 29165


In [10]:
# save model
import warnings
warnings.filterwarnings("ignore")
model.save('3_word2vec.mdl')
model.wv.save_word2vec_format('3_word2vec.bin', binary=True)

In [11]:
# exctract word2vec vectors
#import spacy
#nlp = spacy.load('en')
#model.wv.get_vector('trump')
#vecs1 = [doc.vector for doc in nlp.pipe(train_df['question1'], n_threads=50)]
#vecs1 =  np.array(vecs1)
#train_df['q1_feats'] = list(vecs1)
    
#vecs2 = [doc.vector for doc in nlp.pipe(train_df['question2'], n_threads=50)]
#vecs2 =  np.array(vecs2)
#train_df['q2_feats'] = list(vecs2)

    # save features
#pd.to_pickle(df, '1_df.pkl')

array([ 2.55013183e-02, -1.99332628e-02, -7.53262788e-02,  6.20045997e-02,
        9.39455815e-03,  1.66408401e-02, -8.34304467e-02,  2.60214731e-02,
        8.20631236e-02, -3.71444039e-02,  1.41054792e-02,  6.90248981e-02,
       -4.04791683e-02, -9.00394619e-02, -8.32098275e-02,  6.78742975e-02,
        6.94911033e-02,  5.17239049e-02, -5.29200025e-02, -5.43501228e-03,
       -8.29669163e-02, -1.05703585e-02, -5.34159653e-02,  2.92051602e-02,
        7.55879730e-02,  3.08486144e-03,  3.87061648e-02, -1.48168877e-02,
       -6.17665015e-02,  4.35097069e-02, -3.85240861e-03, -3.01070809e-02,
        5.70424125e-02,  9.67636257e-02, -9.30578485e-02, -5.58524542e-02,
        4.13453989e-02, -9.51981321e-02,  9.75056663e-02,  6.16287487e-03,
       -8.86876732e-02,  9.18540135e-02, -2.71328725e-02, -5.80861680e-02,
       -5.28051294e-02,  1.18145337e-02,  7.45410547e-02,  3.13682407e-02,
        5.02458699e-02, -2.97917239e-03, -6.67855814e-02, -7.82001913e-02,
       -5.69902770e-02, -

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(train_df['question1']) + list(train_df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [16]:
#word2tfidf
#w2v.keys()

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [18]:
train_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [19]:
# spacy를 대체함
from konlpy.tag import Twitter 

In [7]:
twitter = Twitter()
twitter.pos("What is the story of Kohinoor (Koh-i-Noor) Diamond?")

NameError: name 'Twitter' is not defined

In [ ]:
# exctract word2vec vectors
#import spacy
#nlp = spacy.load('en')

vecs1 = []
for qu in tqdm(list(train_df['question1'])):
    doc = twitter.pos(qu) 
    mean_vec = np.zeros([len(doc), 300])
    for word in doc:
        # print(word)
        # word2vec
        
        try:
            vec = model.wv.get_vector(word[0])
        except KeyError as ex:
            #print(ex)
            continue
            
        # fetch df score
        
        try:
            idf = word2tfidf[str(word)]
        except:
            #print word
            idf = 0
            
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
    vecs1.append(mean_vec)
train_df['q1_feats'] = list(vecs1)
train_df['q1_feats']

In [ ]:

vecs2 = []
for qu in tqdm(list(train_df['question2'])):
    doc = twitter.pos(qu) 
    mean_vec = np.zeros([len(doc), 300])
    for word in doc:
        # print(word)
        # word2vec
        
        try:
            vec = model.wv.get_vector(word[0])
        except KeyError as ex:
            #print(ex)
            continue
            
        # fetch df score
        
        try:
            idf = word2tfidf[str(word)]
        except:
            #print word
            idf = 0
            
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
    vecs2.append(mean_vec)
train_df['q2_feats'] = list(vecs2)
train_df['q2_feats']

In [46]:
qu = "안녕하세요. 딥러닝 초보자입니다"
doc = twitter.pos(qu) 
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_feats,q2_feats
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
doc

[('안녕하세', 'Adjective'),
 ('요', 'Eomi'),
 ('.', 'Punctuation'),
 ('딥', 'Noun'),
 ('러닝', 'Noun'),
 ('초보자', 'Noun'),
 ('입니', 'Adjective'),
 ('다', 'Eomi')]

In [ ]:
# shuffle df
train_df = train_df.reindex(np.random.permutation(train_df.index))

# set number of train and test instances
num_train = int(train_df.shape[0] * 0.88)
num_test = train_df.shape[0] - num_train                 
print("Number of training pairs: %i"%(num_train))
print("Number of testing pairs: %i"%(num_test))

# init data data arrays
X_train = np.zeros([num_train, 2, 300])
X_test  = np.zeros([num_test, 2, 300])
Y_train = np.zeros([num_train]) 
Y_test = np.zeros([num_test])

# format data 
b = [a[None,:] for a in list(train_df['q1_feats'].values)]
q1_feats = np.concatenate(b, axis=0)

b = [a[None,:] for a in list(train_df['q2_feats'].values)]
q2_feats = np.concatenate(b, axis=0)

# fill data arrays with features
X_train[:,0,:] = q1_feats[:num_train]
X_train[:,1,:] = q2_feats[:num_train]
Y_train = train_df[:num_train]['is_duplicate'].values
            
X_test[:,0,:] = q1_feats[num_train:]
X_test[:,1,:] = q2_feats[num_train:]
Y_test = train_df[num_train:]['is_duplicate'].values

# remove useless variables
del b
del q1_feats
del q2_feats

In [ ]:
from siamese import *
X_train[0]

In [ ]:
# create model

from siamese import *
from keras.optimizers import RMSprop, SGD, Adam
net = create_network(300)

# train
#optimizer = SGD(lr=1, momentum=0.8, nesterov=True, decay=0.004)
optimizer = Adam(lr=0.001)
net.compile(loss=contrastive_loss, optimizer=optimizer)

for epoch in range(50):
    net.fit([X_train[:,0,:], X_train[:,1,:]], Y_train,
          validation_data=([X_test[:,0,:], X_test[:,1,:]], Y_test),
          batch_size=128, nb_epoch=1, shuffle=True, )
    
    # compute final accuracy on training and test sets
    pred = net.predict([X_test[:,0,:], X_test[:,1,:]], batch_size=128)
    te_acc = compute_accuracy(pred, Y_test)
    
#    print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
    print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))